<a href="https://colab.research.google.com/github/vadhri/ai-notebook/blob/main/Notebooks/Chap20/20_2_Full_Batch_Gradient_Descent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Notebook 20.2: Full Batch Gradient Descent**

This notebook investigates training a network with full batch gradient descent as in figure 20.2.  There is also a version (notebook takes a long time to run), but this didn't speed it up much for me.  If you run out of CoLab time,  you'll need to download the Python file and run locally.

Work through the cells below, running each cell in turn. In various places you will see the words "TO DO". Follow the instructions at these places and make predictions about what is going to happen or write code to complete the functions.

Contact me at udlbookmail@gmail.com if you find any mistakes or have any suggestions.

In [1]:
# Run this if you're in a Colab to install MNIST 1D repository
!pip install git+https://github.com/greydanus/mnist1d

  Cloning https://github.com/greydanus/mnist1d to /tmp/pip-req-build-ulpfzbwt
  Running command git clone --filter=blob:none --quiet https://github.com/greydanus/mnist1d /tmp/pip-req-build-ulpfzbwt
  Resolved https://github.com/greydanus/mnist1d to commit 7878d96082abd200c546a07a4101fa90b30fdf7e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mnist1d: filename=mnist1d-0.0.2.post16-py3-none-any.whl size=14628 sha256=c96052edf7babf7e2a885e81386d8d5ba9ad81ffdfc3b88a61273353da7ef506
  Stored in directory: /tmp/pip-ephem-wheel-cache-_tehsva9/wheels/41/95/55/45fcf7d38cfa97e35682da6c5017d4513729131326ce935371
Successfully built mnist1d


In [2]:
import numpy as np
import os
import torch, torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from torch.optim.lr_scheduler import StepLR
import matplotlib.pyplot as plt
import mnist1d
import random
from IPython.display import display, clear_output

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [29]:
args = mnist1d.data.get_dataset_args()
data = mnist1d.data.get_dataset(args, path='./mnist1d_data.pkl', download=False, regenerate=False)
print(data.keys())
for k in data.keys():
  if k != 'templates':
    data[k] = torch.tensor(data[k], dtype=torch.float64).to(device)

# The training and test input and outputs are in
# data['x'], data['y']
print("Examples in training set: {}".format(len(data['y'])))
print("Length of each example: {}".format(data['x'].shape[-1]))

Successfully loaded data from ./mnist1d_data.pkl
dict_keys(['x', 'x_test', 'y', 'y_test', 't', 'templates'])
Examples in training set: 4000
Length of each example: 40


Define the network

In [30]:
# Data is length forty, and there are 10 classes
D_i = 40
D_o = 10

# create model with one hidden layer and 298 hidden units
model_1_layer = nn.Sequential(
nn.Linear(D_i, 298),
nn.ReLU(),
nn.Linear(298, D_o)).to(device)


# TODO -- create model with three hidden layers and 100 hidden units per layer

model_2_layer = nn.Sequential(
  nn.Linear(D_i, 100),
  nn.ReLU(),
  nn.Linear(100, 100),
  nn.ReLU(),
  nn.Linear(100, 100),
  nn.ReLU(),
  nn.Linear(100, D_o)
).to(device)



# TODO -- Create model with three hidden layers and 75 hidden units per layer
# Replace this line
model_3_layer = nn.Sequential(
  nn.Linear(D_i, 75),
  nn.ReLU(),
  nn.Linear(75, 75),
  nn.ReLU(),
  nn.Linear(75, 75),
  nn.ReLU(),
  nn.Linear(75, D_o)
).to(device)




# TODO create model with four hidden layers and 63 hidden units per layer
# Replace this line
model_4_layer = nn.Sequential(
  nn.Linear(D_i, 63),
  nn.ReLU(),
  nn.Linear(63, 63),
  nn.ReLU(),
  nn.Linear(63, 63),
  nn.ReLU(),
  nn.Linear(63, 63),
  nn.ReLU(),
  nn.Linear(63, D_o)
).to(device)

In [31]:
# He initialization of weights
def weights_init(layer_in):
  if isinstance(layer_in, nn.Linear):
    nn.init.kaiming_uniform_(layer_in.weight)
    layer_in.bias.data.fill_(0.0)

In [43]:
def train_model(model, train_data_x, train_data_y, n_epoch):
  print("This is going to take a long time!")
  # choose cross entropy loss function (equation 5.24 in the loss notes)
  loss_function = nn.CrossEntropyLoss()
  # construct SGD optimizer and initialize learning rate to small value and momentum to 0
  optimizer = torch.optim.SGD(model.parameters(), lr = 0.0025, momentum=0.0)
  # create 100 dummy data points and store in data loader class
  x_train = torch.tensor(train_data_x.t(), dtype=torch.float32)
  y_train = torch.tensor(train_data_y, dtype=torch.long)

  # load the data into a class that creates the batches -- full batch as there are 4000 examples
  data_loader = DataLoader(TensorDataset(x_train,y_train), batch_size=4000, shuffle=False, worker_init_fn=np.random.seed(1))

  # Initialize model weights
  model.apply(weights_init)

  # store the errors percentage at each point
  errors_train = np.zeros((n_epoch))

  for epoch in range(n_epoch):
    # loop over batches
    for i, data in enumerate(data_loader):
      # retrieve inputs and labels for this batch
      x_batch, y_batch = data
      # zero the parameter gradients
      optimizer.zero_grad()
      # forward pass -- calculate model output
      pred = model(x_batch)
      # compute the loss
      loss = loss_function(pred, y_batch)
      # Store the errors
      _, predicted_train_class = torch.max(pred.data, 1)
      errors_train[epoch] = 100 - 100 * (predicted_train_class == y_train).float().sum() / len(y_train)
      # backward pass
      loss.backward()
      # SGD update
      optimizer.step()

      if epoch % 10 == 0:
        clear_output(wait=True)
        display("Epoch %d, errors_train %3.3f"%(epoch, errors_train[epoch]))

  return errors_train

In [41]:
# Load in the data
train_data_x = data['x'].t()
train_data_y = data['y']
# Print out sizes
print("Train data: %d examples (columns), each of which has %d dimensions (rows)"%((train_data_x.shape[1],train_data_x.shape[0])))

Train data: 4000 examples (columns), each of which has 40 dimensions (rows)


In [ ]:
# Train the models
errors_four_layers = train_model(model_4_layer, train_data_x, train_data_y, n_epoch=200000)

'Epoch 1180, errors_train 70.525'

In [ ]:
errors_three_layers = train_model(model_3_layer, train_data_x, train_data_y, n_epoch=200000)


In [ ]:
errors_two_layers = train_model(model_2_layer, train_data_x, train_data_y, n_epoch=200000)


In [ ]:
errors_one_layer = train_model(model_1_layer, train_data_x, train_data_y, n_epoch=500000)

In [ ]:
# Plot the results
fig, ax = plt.subplots()
ax.plot(errors_one_layer,'r-',label='one layer')
ax.plot(errors_two_layers,'g-',label='two layers')
ax.plot(errors_three_layers,'b-',label='three layers')
ax.plot(errors_four_layers,'m-',label='four layers')
ax.set_ylim(0,100)
ax.set_xlabel('Epoch'); ax.set_ylabel('Percent error')
ax.legend()
plt.show()